In [1]:
using JuMP,Ipopt
include("Functions/Conor_adjoint.jl");

Z=0.01+0.1*im
Y=inv(Z)
g=real(Y)
b=imag(Y)

myDict=Dict()

myDict["g"]=g
myDict["b"]=b
myDict["V_max"]=1.1
myDict["V_min"]=0.9
myDict["Pload"]=1
myDict["Qload"]=0
myDict["S_pv"]=0.5

# Initial Guess for NR powerflow
myDict["Var"]=Dict()
myDict["Var"]["V2"]=1
myDict["Var"]["T2"]=0

0

In [10]:
model = Model()
     # register functions and derivatives to model
    JuMP.register(model, :Voltage2, 3, Voltage2,diffVoltage2)
    JuMP.register(model, :Pinj_inner, 3, Pinj_func,Pinj_func_diff)
    JuMP.register(model, :test, 1, test1, autodiff=true)

    @variable(model, myDict["V_min"]<=V1<=myDict["V_max"])
    @variable(model, P_PV)
    @variable(model, Q_PV)
    @variable(model, x)

    @NLconstraint(model, P_PV^2+Q_PV^2 <= myDict["S_pv"])
    @NLconstraint(model,myDict["V_min"]<= Voltage2(V1,P_PV,Q_PV))


0.9 - Voltage2(V1, P_PV, Q_PV) <= 0

In [49]:
using JuMP,Ipopt
include("Functions/Conor_adjoint.jl");

model = Model(with_optimizer(Ipopt.Optimizer,warm_start_init_point="yes",print_level=0));
# JuMP.register(model, :test1, 1, test, test_diff,test_diff2)
JuMP.register(model, :test1, 1, test, autodiff=true )

@variable(model, x, start = -1)
@constraint(model, -1<=x<=1)
@NLconstraint(model, test1(x) == 0)
@NLobjective(model, Min, x)
optimize!(model); stat = string(termination_status(model))

"LOCALLY_SOLVED"

In [50]:
JuMP.value(x)

-1.0

In [42]:
model

A JuMP Model
Minimization problem with:
Variable: 1
Objective function type: Nonlinear
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Nonlinear: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.
Names registered in the model: x